# 🪿 Goose System - Fine-Tuning

Fine-tune Phi-3-mini on HiveLab tool generation data using LoRA.

**Runtime:** GPU (T4) - Go to Runtime > Change runtime type > T4 GPU

**Time:** ~15-30 minutes for 389 examples

## 1. Setup Environment

In [ ]:
# Install Unsloth (4x faster fine-tuning)
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps trl peft accelerate bitsandbytes

In [ ]:
# Verify GPU
import torch
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 2. Upload Training Data

Upload your `combined-training.jsonl` file from:
`packages/core/src/hivelab/goose/training/data/combined-training.jsonl`

In [ ]:
from google.colab import files
import json

# Upload training data
print("Upload combined-training.jsonl:")
uploaded = files.upload()

# Load examples
training_file = list(uploaded.keys())[0]
examples = []
with open(training_file, 'r') as f:
    for line in f:
        if line.strip():
            examples.append(json.loads(line))

print(f"\nLoaded {len(examples)} training examples")

## 3. Configure Model

In [ ]:
from unsloth import FastLanguageModel

# Model configuration
max_seq_length = 2048
dtype = None  # Auto-detect
load_in_4bit = True  # Use 4-bit quantization

# Load Phi-3 mini
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Phi-3-mini-4k-instruct",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

print("Model loaded!")

In [ ]:
# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # LoRA rank
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
)

print("LoRA adapters added!")

## 4. Prepare Training Data

In [ ]:
# System prompt for Goose
SYSTEM_PROMPT = """You are Goose, an AI that generates HiveLab tools from natural language descriptions.

OUTPUT FORMAT:
Output valid JSON: {"elements":[{"type":"...","instanceId":"...","config":{...},"position":{"x":100,"y":100},"size":{"width":280,"height":200}}],"connections":[],"name":"...","description":"...","layout":"grid"}

RULES:
1. Output valid JSON only - no explanations
2. Valid elements: poll-element, rsvp-button, countdown-timer, leaderboard, chart-display, result-list, form-builder, counter, timer, search-input, filter-selector, date-picker
3. Required: poll-element needs question+options, rsvp-button needs eventName, countdown-timer needs targetDate, chart-display needs chartType
4. Keep tools simple - 1-4 elements max"""

def format_example(example):
    """Format for Phi-3 instruction tuning."""
    prompt = example["prompt"]
    output = json.dumps(example["output"], separators=(',', ':'))

    return f"""<|system|>
{SYSTEM_PROMPT}
<|end|>
<|user|>
{prompt}
<|end|>
<|assistant|>
{output}
<|end|>"""

# Format all examples
formatted_data = [{"text": format_example(ex)} for ex in examples]

print(f"Formatted {len(formatted_data)} examples")
print(f"\nSample (truncated):\n{formatted_data[0]['text'][:500]}...")

In [ ]:
from datasets import Dataset

# Create HuggingFace dataset
dataset = Dataset.from_list(formatted_data)
print(f"Dataset: {dataset}")

## 5. Train Model

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=42,
        output_dir="goose-outputs",
    ),
)

print("Trainer configured. Starting training...")

In [ ]:
# Train!
trainer_stats = trainer.train()

print(f"\n{'='*50}")
print("Training complete!")
print(f"{'='*50}")

## 6. Test Inference

In [ ]:
# Enable inference mode
FastLanguageModel.for_inference(model)

def generate_tool(prompt):
    """Generate a HiveLab tool from a prompt."""
    formatted = f"""<|system|>
{SYSTEM_PROMPT}
<|end|>
<|user|>
{prompt}
<|end|>
<|assistant|>
"""
    inputs = tokenizer(formatted, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        temperature=0.3,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    # Extract assistant response
    if "<|assistant|>" in response:
        response = response.split("<|assistant|>")[-1]
    if "<|end|>" in response:
        response = response.split("<|end|>")[0]
    return response.strip()

# Test prompts
test_prompts = [
    "create a poll about favorite study spots",
    "event signup with countdown timer",
    "feedback form with results chart",
]

print("Testing inference...\n")
for prompt in test_prompts:
    print(f"Prompt: {prompt}")
    print("-" * 40)
    result = generate_tool(prompt)
    print(f"Output: {result[:300]}...\n")

    # Validate JSON
    try:
        parsed = json.loads(result)
        print(f"✓ Valid JSON with {len(parsed.get('elements', []))} elements\n")
    except:
        print("✗ Invalid JSON\n")

## 7. Export to GGUF (for Ollama)

In [ ]:
# Save in GGUF format for Ollama
model.save_pretrained_gguf(
    "goose-model",
    tokenizer,
    quantization_method="q4_k_m",  # Good balance of size/quality
)

print("GGUF model saved!")

In [ ]:
# Download the model
from google.colab import files
import glob

# Find the GGUF file
gguf_files = glob.glob("goose-model/*.gguf")
if gguf_files:
    print(f"Downloading: {gguf_files[0]}")
    files.download(gguf_files[0])
else:
    print("GGUF file not found. Check goose-model/ directory.")
    !ls -la goose-model/

## 8. Next Steps

After downloading the GGUF file:

1. **Install Ollama** (if not already): https://ollama.ai

2. **Create the model:**
   ```bash
   # Move GGUF to scripts/goose/
   mv ~/Downloads/goose-model-*.gguf scripts/goose/goose-model.gguf

   # Create Ollama model
   cd scripts/goose
   ollama create goose -f Modelfile
   ```

3. **Test:**
   ```bash
   ollama run goose "create a poll about favorite foods"
   ```

4. **Use in HIVE:**
   The API at `/api/tools/generate` will automatically use Goose!